In [2]:
import pandas as pd 
import numpy as np
from collections import Counter
import os
import unidecode

In [3]:
os.chdir('./data')

In [4]:
ls

cdsp_legi1997t2_circ.xlsx*  cdsp_legi2012t2_circ.xlsx*
cdsp_legi2002t2_circ.xls*   dataframe_elections.xlsx*
cdsp_legi2007t2_circ.xls*


In [83]:
legi_1997 = pd.read_excel("cdsp_legi1997t2_circ.xlsx")
legi_2002 = pd.read_excel("cdsp_legi2002t2_circ.xls")
legi_2007 = pd.read_excel("cdsp_legi2007t2_circ.xls")
legi_2012 = pd.read_excel("cdsp_legi2012t2_circ.xlsx")

In [84]:
legi_tab = pd.read_excel("dataframe_elections.xlsx")

In [85]:
def func_elu(a, b, c):
    if np.isnan(a) == True:
        return 'N'
    voix = a
    voix_max = max([a,b,c])
    if a < voix_max :
        return 'N'
    else :
        return 'O'

In [86]:
# on adapte les columns sur legi_2012 pour qu'elle supporte notre fonction
legi_2012["1 Elu"] = legi_2012.apply(lambda x : func_elu(x['1 voix'], x['2 voix'], x['3 voix']), axis=1)
legi_2012["2 Elu"] = legi_2012.apply(lambda x : func_elu(x['2 voix'], x['1 voix'], x['3 voix']), axis=1)
legi_2012["3 Elu"] = legi_2012.apply(lambda x : func_elu(x['3 voix'], x['2 voix'], x['1 voix']), axis=1)

In [87]:
# exemple sur 97
# get name
def in_format(data, annee):
    name = pd.melt(data, id_vars=["Code département", "département",'circonscription', "Exprimés"], 
            value_vars=["1 Nom candidat","2 Nom candidat","3 Nom candidat"], value_name="name")
    prenom = pd.melt(data,
            value_vars=["1 Prénom candidat","2 Prénom candidat","3 Prénom candidat"], value_name="first_name")
    value = pd.melt(data,
            value_vars=["1 voix","2 voix","3 voix"], value_name="voix")
    elu = pd.melt(data, 
            value_vars=["1 Elu","2 Elu","3 Elu"], value_name="elu")
    df = pd.concat([name, prenom, value, elu], axis=1)

    del df["variable"]
    df["score"] = df.apply(lambda x : x['voix']/ x['Exprimés'], axis=1)
    df["an"] = annee

    df.dropna(inplace=True) 
    print(df.shape)
    return df

In [88]:
legi_1997 = in_format(legi_1997, 1997)
legi_2002 = in_format(legi_2002, 2002)
legi_2007 = in_format(legi_2007, 2007)
legi_2012 = in_format(legi_2012, 2012)

(1197, 10)
(1045, 10)
(933, 10)
(1101, 10)


In [89]:
legi_2t = pd.concat([legi_1997, legi_2002, legi_2007, legi_2012], axis=0)
legi_2t["code"] = legi_2t.apply(lambda x: str(x["Code département"])+'|'+str(int(x["circonscription"])), axis=1)

In [90]:
legi_2t.shape

(4276, 11)

In [94]:
legi_2t.sort_values(["code","name","first_name"]).head(2)

,Code département,département,circonscription,Exprimés,name,first_name,voix,elu,score,an,code
32,10,AUBE,1.0,30564.0,BRET,LINE,13252.0,N,0.433582,2002,10|1
23,10,AUBE,1.0,32500.0,BRET,Line,12793.0,N,0.393631,2007,10|1


In [95]:
legi_2t["name"] =  legi_2t["name"].map(lambda x: unidecode.unidecode(x))
legi_2t["first_name"] =  legi_2t["first_name"].map(lambda x: unidecode.unidecode(x).upper())
legi_tab["prenom"] =  legi_tab["prenom"] .map(lambda x: unidecode.unidecode(x).upper())
legi_tab["nom"] =  legi_tab["nom"].map(lambda x: unidecode.unidecode(x))

In [105]:
# création du lag second tour legislative
test = pd.merge(legi_2t, legi_2t[["code","name","first_name","score","an"]]
                ,how="left", on=["code","name","first_name"])

test["lag_2nd_tour"] = test.apply(lambda x: x["score_y"] if x["an_x"]==(x["an_y"]+5) else np.nan, axis=1)
test = test[["code","name","first_name","an_x","lag_2nd_tour"]].dropna()
legi_2t = pd.merge(legi_2t, test, how="left", 
                    left_on=["code","name","first_name","an"], right_on=["code","name","first_name","an_x"])
del legi_2t["an_x"]

In [108]:
legi_2t.sort_values(["code","name","first_name","an"]).head(30)

,Code département,département,circonscription,Exprimés,name,first_name,voix,elu,score,an,code,lag_2nd_tour
1229,10,AUBE,1.0,30564.0,BRET,LINE,13252.0,N,0.433582,2002,10|1,NaN
2265,10,AUBE,1.0,32500.0,BRET,LINE,12793.0,N,0.393631,2007,10|1,0.433582
34,10,AUBE,1.0,38770.0,BRET,MARC,15112.0,N,0.389786,1997,10|1,NaN
2732,10,AUBE,1.0,32500.0,DHUICQ,NICOLAS,19707.0,O,0.606369,2007,10|1,NaN
3746,10,AUBE,1.0,38753.0,DHUICQ,NICOLAS,17122.0,O,0.441824,2012,10|1,0.606369
3205,10,AUBE,1.0,38753.0,GAUDOT,RENE,12406.0,N,0.320130,2012,10|1,NaN
599,10,AUBE,1.0,38770.0,MICAUX,PIERRE,17003.0,O,0.438561,1997,10|1,NaN
1748,10,AUBE,1.0,30564.0,MICAUX,PIERRE,17312.0,O,0.566418,2002,10|1,0.438561
1122,10,AUBE,1.0,38770.0,SUBTIL,BRUNO,6655.0,N,0.171653,1997,10|1,NaN
4245,10,AUBE,1.0,38753.0,SUBTIL,BRUNO,9225.0,N,0.238046,2012,10|1,NaN


In [14]:
df_legi = pd.merge(legi_tab, legi_2t,left_on=["nom", "code", "an"], 
                   right_on=["name", "code", "an"])

*On merge sur le code plutot que nom prénom car trop de différence entre les deux tables
en esperant qu'il n'y ait qu'un candidat avec le même nom par code en une date donée*

In [15]:
df_legi.shape

(4293, 73)

In [16]:
# On personalise sur les quelques cas 
legi_2t.ix[(legi_2t["name"]=="PERRIN GAILLARD") & (legi_2t["an"]==2002), "name"] = "GAILLARD"
legi_2t.ix[(legi_2t["name"]=="GAAA") & (legi_2t["an"]==2002), "name"] = "GAIA"
legi_2t.ix[(legi_2t["name"]=="BACHELOT- NARQUIN") & (legi_2t["an"]==2002), "name"] = "BACHELOT-NARQUIN"
legi_2t.ix[(legi_2t["name"]=="VILLAUMe") & (legi_2t["an"]==2007), "name"] = "VILLAUME"
legi_2t.ix[(legi_2t["name"]=="CHEVeNEMENT") & (legi_2t["an"]==2007), "name"] = "CHEVENEMENT"
legi_2t.ix[(legi_2t["name"]=="GUeGOT") & (legi_2t["an"]==2007), "name"] = "GUEGOT"

In [17]:
# liste des nom et first name qui merge mal sur le inner
df_legi = pd.merge(legi_tab, legi_2t,how="right" ,left_on=["nom", "code", "an"],
                   right_on=["name", "code", "an"])
df_legi[~df_legi.nom.notnull()][["name","first_name","an", "circonscription","Code département"]]

,name,first_name,an,circonscription,Code département


In [18]:
df_legi.shape

(4299, 73)

*Il nous manque personne mais on a des doublons vu la dim*  
regarder doublon même année, meme code, et meme nom

In [19]:
test = df_legi.sort_values(["name", "code", "an"])
test["name_lag"] = test["name"].shift(1)
test["code_lag"] = test["code"].shift(1)
test["an_lag"] = test["an"].shift(1)

In [20]:
test = test[(test['name']==test["name_lag"])&(test['code']==test["code_lag"])&(test['an']==test["an_lag"])]
pb_name = test["name"].unique()

In [21]:
#On fait un merge classique sur une table qui ne continet pas pb_name
df_legi_1 = pd.merge(legi_tab[~legi_tab["nom"].isin(pb_name)],
                     legi_2t[~legi_2t["name"].isin(pb_name)],
                     left_on=["nom", "code", "an"], 
                     right_on=["name", "code", "an"])

#puis merge avec prenom sur pb_name
df_legi_2 = pd.merge(legi_tab[legi_tab["nom"].isin(pb_name)],
                     legi_2t[legi_2t["name"].isin(pb_name)],
                     left_on=["nom","prenom", "code", "an"], 
                     right_on=["name","first_name", "code", "an"])

In [22]:
print(df_legi_1.shape)
print(df_legi_2.shape)

(4222, 73)
(51, 73)


*Il manque 3 candidats*

In [24]:
df_legi_2 = pd.merge(legi_tab[legi_tab["nom"].isin(pb_name)],
                     legi_2t[legi_2t["name"].isin(pb_name)],
                     how="right",
                     left_on=["nom","prenom", "code", "an"], 
                     right_on=["name","first_name", "code", "an"])

df_legi_2[~df_legi_2.nom.notnull()][["name","first_name","an", "circonscription","Code département"]]

,name,first_name,an,circonscription,Code département
51,ROCCA SERRA DE,JEAN PAUL,1997,2.0,2A
52,BERNARD,JEAN LOUIS,1997,3.0,45
53,BERNARD,J. LOUIS,2002,3.0,45


In [33]:
legi_tab[(legi_tab["nom"]=="BERNARD")&(legi_tab["c_dep"]==45)]

,an,c_dep,dep,circo,code,inscrits,etrangers,part_impose,revenus_q1,revenus_med,...,sexe,nom,prenom,etiquette,nuance,nuance_groupe,bloc,voix,p_voix,second_tour
18260,2007,45,LOIRET,3,45|3,89957.0,0.059232,0.700041,13464.446268,18679.983327,...,M,BERNARD,JEAN-LOUIS,UNION POUR UN MOUVEMENT POPULAIRE,UMP,LR,Droite,16453.0,0.182898,O
26283,2002,45,LOIRET,3,45|3,83338.0,0.059232,0.696305,12027.746713,16849.008653,...,M,BERNARD,JEAN-LOUIS,SANS ETIQUETTE,UMP,LR,Droite,23229.0,0.278732,O
33601,1997,45,LOIRET,3,45|3,NaN,NaN,NaN,NaN,NaN,...,NaN,BERNARD,JEAN-LOUIS,UDF-RADICAL,UDF,MDM,Centre,16437.0,0.210305,O


In [34]:
legi_2t.head()

,Code département,département,circonscription,Exprimés,name,first_name,voix,elu,score,an,code
0,1,AIN,1.0,51081.0,GODIN,ANDRE,25685.0,O,0.502829,1997,1|1
1,1,AIN,2.0,52285.0,BOLLINI,JOCELYNE,19778.0,N,0.378273,1997,1|2
2,1,AIN,3.0,45603.0,GILBERT,ERIC,20333.0,N,0.445870,1997,1|3
3,1,AIN,4.0,57749.0,RAYMOND,MICHEL,25455.0,N,0.440787,1997,1|4
4,2,AISNE,1.0,49404.0,DOSIERE,RENE,29331.0,O,0.593697,1997,2|1


In [40]:
# On modifie manuellement
legi_2t.ix[(legi_2t["name"]=="ROCCA SERRA DE")&(legi_2t["first_name"]=="JEAN PAUL"), "first_name"] = "JEAN-PAUL"
legi_2t.ix[(legi_2t["name"]=="BERNARD")&(legi_2t["Code département"]==45),"first_name"] = "JEAN-LOUIS"

In [41]:
#On fait un merge classique sur une table qui ne continet pas pb_name
df_legi_1 = pd.merge(legi_tab[~legi_tab["nom"].isin(pb_name)],
                     legi_2t[~legi_2t["name"].isin(pb_name)],
                     left_on=["nom", "code", "an"], 
                     right_on=["name", "code", "an"])

#puis merge avec prenom sur pb_name
df_legi_2 = pd.merge(legi_tab[legi_tab["nom"].isin(pb_name)],
                     legi_2t[legi_2t["name"].isin(pb_name)],
                     left_on=["nom","prenom", "code", "an"], 
                     right_on=["name","first_name", "code", "an"])

print(df_legi_1.shape)
print(df_legi_2.shape)
print(df_legi_1.shape[0]+df_legi_2.shape[0])

(4222, 73)
(54, 73)
4276


In [65]:
df_legi = pd.concat([df_legi_1, df_legi_2], axis=0)

In [45]:
df_legi.columns

Index(['an', 'c_dep', 'dep', 'circo', 'code', 'inscrits', 'etrangers',
       'part_impose', 'revenus_q1', 'revenus_med', 'revenus_q3',
       'ecart_revenus', 'chom_tot', 'chom_tot_evol_5', 'chom_jeunes',
       'chom_jeunes_evol_5', 'chom_adultes', 'chom_adultes_evol_5',
       'chom_seniors', 'chom_seniors_evol_5', 'p_agri', 'p_commercants',
       'p_cadres', 'p_intermed', 'p_employes', 'p_ouvriers', 'd_brevet',
       'd_bep', 'd_bac', 'd_sup', 'circo_parti', 'circo_nuance',
       'circo_nuance_groupe', 'circo_bloc', 'circo_leg_meme_nuance',
       'nb_candidats_meme_bloc', 'score_nuance_groupe_prec_leg',
       'score_bloc_prec_leg', 'score_leg_exg', 'score_leg_g', 'score_leg_c',
       'score_leg_d', 'score_leg_exd', 'circo_nuance_groupe_pres',
       'circo_pres_meme_nuance', 'score_nuance_groupe_pres', 'score_bloc_pres',
       'score_pres_exg', 'score_pres_g', 'score_pres_c', 'score_pres_d',
       'score_pres_exd', 'score_candidat_prec_leg', 'depute_sortant', 'sexe',
      

In [76]:
columns_to_keep =['an','Code département', 'département',
       'circonscription', 'code','name', 'first_name',"sexe",
        "circo_bloc","circo_leg_meme_nuance","circo_pres_meme_nuance",
        "score_bloc_pres","depute_sortant", "bloc", "p_voix",
         'voix_y','Exprimés', 'score','elu',
]

**creer:**  
* une variable d'oppostion selon nuance un peu commme la variable candidat du même bloc  
* score_nuance_groupe_prec_leg tour   
* score_bloc_prec_leg  tour
* score leg de l'adversaire au second tour
* score au 1er tour du groupe et du candidat 
* eventuellement score du groupe pres au second tour des présidentielles
* score du groupe du candidat dans la circo 
* score candidat au second tour d'une précédente législative 
* ~~triangulaire~~ 

In [77]:
df_legi_f = df_legi[columns_to_keep]

In [78]:
df_legi_f.head()

,an,Code département,département,circonscription,code,name,first_name,sexe,circo_bloc,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,bloc,p_voix,voix_y,Exprimés,score,elu
0,2012,1,AIN,1.0,1|1,BRETON,XAVIER,M,Droite,1,1.0,0.257037,1.0,Droite,0.224066,24219.0,46962.0,0.515715,O
1,2012,1,AIN,1.0,1|1,DEBAT,JEAN-FRANCOIS,M,Droite,0,0.0,0.291068,0.0,Gauche,0.231604,22743.0,46962.0,0.484285,N
2,2012,1,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,M,Droite,1,1.0,0.275342,1.0,Droite,0.220931,22327.0,50383.0,0.443146,O
3,2012,1,AIN,2.0,1|2,EYRAUD,OLIVIER,M,Droite,0,0.0,0.179791,0.0,Exdroite,0.125244,8530.0,50383.0,0.169303,N
4,2012,1,AIN,2.0,1|2,RAYMOND,MICHEL,M,Droite,0,0.0,0.280721,0.0,Gauche,0.136810,19526.0,50383.0,0.387551,N


* triangulaire

In [87]:
triangulaire = df_legi.groupby(["an","code"])["name"].count().reset_index()
triangulaire["top_triangulaire"] = triangulaire["name"].map(lambda x: 1 if x>2 else 0)
del triangulaire["name"]
df_legi_f = pd.merge(df_legi_f, triangulaire, how="left", on=["an","code"])

* lag second tour legislative

In [99]:
lag2t = df_legi_f.groupby(["an","name", "code"])["bloc"].count().reset_index()
lag2t[lag2t["bloc"]>1]

,an,name,code,bloc
1019,1997,ROCCA SERRA DE,2A|2,2


=> on réalise ça lors de la création de la table de score de second tour
comme cela on peut utiliser les prénoms normalisés